# 98. Feature Pyramid Network (FPN)

## 学習目標

1. **マルチスケール特徴**の重要性
2. **FPN**の構造と動作原理
3. **物体検出・セグメンテーション**での活用

## 目次

1. [マルチスケール問題](#section1)
2. [FPNの構造](#section2)
3. [実装](#section3)
4. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)

<a id="section1"></a>
## 1. マルチスケール問題

画像内の物体は様々なサイズで現れます。

In [ ]:
def visualize_multi_scale_problem():
    """マルチスケール問題の可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 様々なサイズの物体を含む画像
    ax = axes[0]
    img = np.ones((100, 100, 3)) * 0.9
    
    # 大きい物体
    img[10:60, 10:50, :] = [0.8, 0.3, 0.3]
    # 中くらいの物体
    img[70:90, 60:80, :] = [0.3, 0.8, 0.3]
    # 小さい物体
    img[30:38, 70:78, :] = [0.3, 0.3, 0.8]
    
    ax.imshow(img)
    ax.set_title('様々なサイズの物体', fontsize=12)
    ax.axis('off')
    
    # 単一スケールの問題
    ax = axes[1]
    ax.text(0.5, 0.8, '単一スケール特徴の問題', fontsize=14, ha='center', fontweight='bold')
    ax.text(0.5, 0.6, '浅い層: 小さい物体に適するが\n意味的情報が弱い', fontsize=11, ha='center')
    ax.text(0.5, 0.35, '深い層: 意味的情報が強いが\n小さい物体を見落とす', fontsize=11, ha='center')
    ax.axis('off')
    
    # FPNの解決策
    ax = axes[2]
    ax.text(0.5, 0.8, 'FPNの解決策', fontsize=14, ha='center', fontweight='bold', color='green')
    ax.text(0.5, 0.5, '全ての解像度で\n強い意味的特徴を持つ\nピラミッドを構築', fontsize=12, ha='center')
    ax.axis('off')
    
    plt.suptitle('マルチスケール問題とFPN', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_multi_scale_problem()

<a id="section2"></a>
## 2. FPNの構造

In [ ]:
def visualize_fpn_architecture():
    """FPNアーキテクチャの可視化"""
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # ボトムアップ（CNN backbone）
    bottom_up = [
        (0.1, 0.8, 0.08, 0.15, 'C1'),
        (0.1, 0.6, 0.07, 0.12, 'C2'),
        (0.1, 0.4, 0.06, 0.09, 'C3'),
        (0.1, 0.2, 0.05, 0.06, 'C4'),
        (0.1, 0.05, 0.04, 0.04, 'C5'),
    ]
    
    # トップダウン + ラテラル接続
    top_down = [
        (0.5, 0.05, 0.04, 0.04, 'P5'),
        (0.5, 0.2, 0.05, 0.06, 'P4'),
        (0.5, 0.4, 0.06, 0.09, 'P3'),
        (0.5, 0.6, 0.07, 0.12, 'P2'),
    ]
    
    # ボトムアップ描画
    for x, y, w, h, label in bottom_up:
        rect = Rectangle((x, y - h/2), w, h, facecolor='lightblue', edgecolor='blue', linewidth=2)
        ax.add_patch(rect)
        ax.text(x + w/2, y, label, ha='center', va='center', fontsize=11)
    
    # トップダウン描画
    for x, y, w, h, label in top_down:
        rect = Rectangle((x, y - h/2), w, h, facecolor='lightgreen', edgecolor='green', linewidth=2)
        ax.add_patch(rect)
        ax.text(x + w/2, y, label, ha='center', va='center', fontsize=11)
    
    # ボトムアップ矢印
    for i in range(len(bottom_up) - 1):
        ax.annotate('', xy=(bottom_up[i+1][0] + bottom_up[i+1][2]/2, bottom_up[i+1][1] + bottom_up[i+1][3]/2),
                   xytext=(bottom_up[i][0] + bottom_up[i][2]/2, bottom_up[i][1] - bottom_up[i][3]/2),
                   arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))
    
    # ラテラル接続（横の矢印）
    laterals = [(bottom_up[4], top_down[0]), (bottom_up[3], top_down[1]),
                (bottom_up[2], top_down[2]), (bottom_up[1], top_down[3])]
    
    for bu, td in laterals:
        ax.annotate('', xy=(td[0], td[1]),
                   xytext=(bu[0] + bu[2], bu[1]),
                   arrowprops=dict(arrowstyle='->', color='orange', lw=2))
    
    # トップダウン矢印
    for i in range(len(top_down) - 1):
        ax.annotate('', xy=(top_down[i+1][0] + top_down[i+1][2]/2, top_down[i+1][1] - top_down[i+1][3]/2),
                   xytext=(top_down[i][0] + top_down[i][2]/2, top_down[i][1] + top_down[i][3]/2),
                   arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # ラベル
    ax.text(0.1, 0.95, 'Bottom-Up\n(Backbone CNN)', ha='center', fontsize=12, fontweight='bold')
    ax.text(0.5, 0.95, 'Top-Down\n(Feature Pyramid)', ha='center', fontsize=12, fontweight='bold')
    ax.text(0.3, 0.5, 'Lateral\nConnections\n(1×1 Conv)', ha='center', fontsize=10, color='orange')
    
    ax.set_xlim(0, 0.7)
    ax.set_ylim(-0.05, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Feature Pyramid Network (FPN)', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_fpn_architecture()

<a id="summary"></a>
## 4. まとめ

### FPNの特徴

1. **マルチスケール特徴**: 全解像度で強い意味的特徴
2. **ボトムアップ + トップダウン**: 詳細情報と意味情報の融合
3. **ラテラル接続**: 1×1畳み込みでチャンネル数を調整

### 応用

- 物体検出（Faster R-CNN + FPN）
- インスタンスセグメンテーション（Mask R-CNN）
- セマンティックセグメンテーション